<a href="https://colab.research.google.com/github/dlwodnr59/-LSTM-model-for-binary-classification/blob/main/lstm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
from PIL import Image
import numpy as np

def resize_images(root_path, output_path, size=(224,224)):
    for dirpath, _, filenames in os.walk(root_path):
        for filename in filenames:
            if filename.endswith('.jpg'):
                filepath = os.path.join(dirpath, filename)
                with Image.open(filepath) as img:
                    img = img.resize(size, Image.BICUBIC)
                    img.save(os.path.join(output_path, filename))

def create_dataset(root_path):
    data = []
    for dirpath, _, filenames in os.walk(root_path):
        for filename in filenames:
            if filename.endswith('.npy'):
                label = int(dirpath[-1])
                filepath = os.path.join(dirpath, filename)
                with open(filepath, 'rb') as f:
                    img = np.load(f)
                    data.append((img, label))
    return data

In [7]:
import torch
from torch.utils.data import DataLoader, Dataset

class SequenceDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sequence, label = self.data[idx]
        return torch.from_numpy(sequence), label

def create_dataloader(data, batch_size=32):
    dataset = SequenceDataset(data)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [9]:
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

def train_model(model, dataloader, optimizer, criterion, device, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()

            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(dataloader.dataset)
        print(f'Epoch {epoch+1} training loss: {epoch_loss:.4f}')

In [2]:
import cv2
import numpy as np
import os

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# 이미지 크기를 맞춰주는 함수
def resize_image(image_path, size=(64, 64)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, size)
    return img.astype(np.float32) / 255.0

# 데이터셋 경로와 시퀸스 길이를 지정합니다.
data_path = 'new'
sequence_length = 20



def load_data(data_path, sequence_length):
    X = []
    y = []
    for label in range(2):
        img_path = f'{data_path}/{label}/img'
        if label == 0:
          for i in range(900, 1093 - sequence_length + 1):
            img_seq = []
            for j in range(i, i+sequence_length):
                img = resize_image(f'{img_path}/img{j}.jpg', size=(64, 64))
                img_seq.append(img)
            X.append(img_seq)
            y.append(label)
        else:
          for i in range(1093, 1148 - sequence_length + 1):
            img_seq = []
            for j in range(i, i+sequence_length):
                img = resize_image(f'{img_path}/img{j}.jpg', size=(64, 64))
                img_seq.append(img)
            X.append(img_seq)
            y.append(label)
        
    return np.array(X), np.array(y)

# 데이터셋 로드
X, y = load_data(data_path, sequence_length)

# 시퀸스 길이 축을 추가하여 X를 reshape 합니다.
X = X.reshape((-1, sequence_length, 64, 64, 3))
print(X.shape)
early_stop = EarlyStopping(monitor='val_loss', patience=5)
# LSTM 모델 정의
model = tf.keras.Sequential([
    layers.Input(shape=(sequence_length, 64, 64, 3)),
    layers.TimeDistributed(layers.Conv2D(32, 3, activation='relu')),
    layers.TimeDistributed(layers.MaxPooling2D(2)),
    layers.TimeDistributed(layers.Flatten()),
    layers.LSTM(64),
    layers.Dense(1, activation='sigmoid')
])

# 모델 컴파일 및 학습
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=30, batch_size=32, validation_split=0.3,callbacks=[early_stop])


(210, 20, 64, 64, 3)
Epoch 1/30
5/5 [==============================] - 22s 4s/step - loss: 0.1895 - accuracy: 0.7823 - val_loss: 2.8545 - val_accuracy: 0.4286
Epoch 2/30
5/5 [==============================] - 21s 4s/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 3.0143 - val_accuracy: 0.4286
Epoch 3/30
5/5 [==============================] - 17s 4s/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 3.1635 - val_accuracy: 0.4286
Epoch 4/30
5/5 [==============================] - 18s 4s/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 3.2285 - val_accuracy: 0.4286
Epoch 5/30
5/5 [==============================] - 17s 4s/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 3.2966 - val_accuracy: 0.4286
Epoch 6/30
5/5 [==============================] - 18s 4s/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 3.3446 - val_accuracy: 0.4286


In [17]:
!pip install CustomDataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement CustomDataset (from versions: none)
ERROR: No matching distribution found for CustomDataset
